<a href="https://colab.research.google.com/github/davidlealo/vocacional-test/blob/main/prototipo_mentoria_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de Gradio en Colab (ejecuta esto una vez)
!pip install gradio

import gradio as gr
import random
import datetime

# Simulaciones de funciones auxiliares (backend processes)
def clasificar_tema(texto):
    # Simulación: Clasifica el tema basado en palabras clave o random para demo
    temas_posibles = ["Biología", "Matemáticas", "Lenguaje", "Historia", "Orientación Vocacional", "Educación Superior"]
    return random.choice(temas_posibles)

def detectar_emocion(texto):
    # Simulación: Detecta emoción basada en palabras o random para demo
    emociones = ["Entusiasmo", "Confusión", "Curiosidad", "Frustración", "Motivación", "Ansiedad"]
    return random.choice(emociones)

def estimar_comprension(texto):
    # Simulación: Estima nivel de comprensión basado en longitud o random para demo
    niveles = ["Alto (80-100%)", "Medio (60-79%)", "Bajo (<60%)"]
    return random.choice(niveles)

def generar_ticket_salida(texto):
    # Simulación: Genera preguntas de ticket de salida relevantes al tema
    preguntas = [
        "1. ¿Qué gas absorben las plantas durante la fotosíntesis?\nA) Oxígeno\nB) Nitrógeno\nC) Dióxido de carbono\nD) Hidrógeno",
        "2. ¿Qué parte de la célula contiene el material genético?\nA) Mitocondria\nB) Núcleo\nC) Ribosoma\nD) Membrana",
        "3. Explica con tus palabras por qué la luz solar es importante para las plantas.",
        "4. ¿Qué fue lo más interesante que aprendiste hoy y por qué?",
        "5. ¿Qué carrera te interesa y por qué? (Basado en la orientación vocacional discutida)"
    ]
    return "\n\n".join(random.sample(preguntas, 3))  # Selecciona 3 preguntas al azar para variedad

# Simulación de almacenamiento (opcional, pero muestra logs)
registro = []
def guardar_conversacion(texto, tema, emocion, comprension, ticket):
    fecha = datetime.datetime.now().isoformat()
    registro.append({
        "fecha": fecha,
        "tema": tema,
        "emocion": emocion,
        "comprension": comprension,
        "ticket": ticket,
        "texto": texto
    })
    # Retorna un log para mostrar en front
    return f"[LOG] Conversación guardada en registro (ID: {len(registro)}). Fecha: {fecha}"

# Función principal: Procesa la entrada y genera salidas (backend logic)
def analizar_conversacion(texto):
    # Paso 1: Clasificar tema
    tema = clasificar_tema(texto)
    log = f"[LOG] Paso 1: Tema clasificado como '{tema}' basado en el texto proporcionado.\n"

    # Paso 2: Detectar emoción
    emocion = detectar_emocion(texto)
    log += f"[LOG] Paso 2: Emoción detectada como '{emocion}' analizando el tono del texto.\n"

    # Paso 3: Estimar comprensión
    comprension = estimar_comprension(texto)
    log += f"[LOG] Paso 3: Nivel de comprensión estimado en '{comprension}' evaluando la complejidad del texto.\n"

    # Paso 4: Generar ticket de salida
    ticket = generar_ticket_salida(texto)
    log += f"[LOG] Paso 4: Ticket de salida generado con preguntas relevantes.\n"

    # Paso 5: Guardar conversación
    save_log = guardar_conversacion(texto, tema, emocion, comprension, ticket)
    log += save_log + "\n"

    # Log final
    log += "[LOG] Proceso completado exitosamente."

    return tema, emocion, comprension, ticket, log

# Interfaz Gradio (frontend)
iface = gr.Interface(
    fn=analizar_conversacion,  # Función backend
    inputs=gr.Textbox(lines=15, label="Transcripción de la conversación"),  # Input del usuario
    outputs=[
        gr.Textbox(label="Tema identificado"),  # Output 1
        gr.Textbox(label="Emoción detectada"),  # Output 2
        gr.Textbox(label="Nivel de comprensión"),  # Output 3
        gr.Textbox(lines=10, label="Ticket de salida generado"),  # Output 4
        gr.Textbox(lines=5, label="Logs del sistema (Proceso backend)")  # Output 5: Muestra los logs del proceso backend en el front
    ],
    title="Analizador de Conversaciones de Mentoría",
    description="Ingresa la transcripción completa de una sesión de mentoría. El sistema clasificará la conversación, detectará emociones, estimará comprensión y generará un ticket de salida. Los logs muestran el proceso backend paso a paso."
)

# Lanzar la app en Colab (con share=True para enlace público)
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e700b124994b231d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
# Instalación de dependencias en Colab
!pip install gradio python-dotenv openai requests

import gradio as gr
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests
import datetime

# === Configuración de Azure (Search y OpenAI) ===
load_dotenv()  # Carga variables de .env (sube tu archivo .env a Colab)

# Validar variables requeridas
required_vars = [
    "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
    "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
]
for var in required_vars:
    if not os.environ.get(var):
        raise ValueError(f"Falta variable de entorno: {var}")

# Configuración de Azure Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Inicializar cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2023-08-01-preview",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

# === Función de Búsqueda en Azure Search (RAG Retrieval) ===
def search_documents(query: str, top_k: int = 5) -> str:
    """Realiza una búsqueda en Azure Cognitive Search y retorna el contexto concatenado."""
    try:
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
        headers = {
            "Content-Type": "application/json",
            "api-key": AZURE_SEARCH_API_KEY
        }
        payload = {
            "search": query,
            "top": top_k
        }
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        results = response.json()
        documents = [doc.get("content", "") for doc in results.get("value", [])]
        return "\n\n".join(documents) if documents else "No se encontraron documentos relevantes."
    except Exception as e:
        return f"Error en búsqueda: {str(e)}"

# === Función de Generación de Respuesta (RAG Generation) ===
def generate_answer(question: str, context: str) -> str:
    """Genera una respuesta usando Azure OpenAI basada en pregunta y contexto de búsqueda."""
    if not context.strip():
        return "No hay contexto suficiente para responder. Intenta reformular la pregunta."

    system_prompt = """
    Eres un mentor pedagógico especializado en orientación vocacional y educación.
    Responde de forma clara, breve y útil, basándote únicamente en el contexto proporcionado.
    Si el contexto es insuficiente, indica que no puedes responder con precisión.
    """
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"
    try:
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error al generar respuesta: {str(e)}"

# === Backend: Funciones de Clasificación con Azure OpenAI ===
def clasificar_tema(transcripcion: str) -> str:
    """Clasifica el tema usando Azure OpenAI."""
    prompt = f"""
    Clasifica el tema principal de esta transcripción de una sesión de mentoría en una de estas categorías:
    Biología, Matemáticas, Lenguaje, Historia, Orientación Vocacional, Educación Superior, Otros.
    Responde solo con el nombre de la categoría.

    Transcripción: {transcripcion}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def detectar_emocion(transcripcion: str) -> str:
    """Detecta la emoción usando Azure OpenAI."""
    prompt = f"""
    Detecta la emoción predominante en esta transcripción de mentoría (Entusiasmo, Confusión, Curiosidad, Frustración, Motivación, Ansiedad).
    Responde solo con el nombre de la emoción.

    Transcripción: {transcripcion}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def estimar_comprension(transcripcion: str) -> str:
    """Estima el nivel de comprensión usando Azure OpenAI."""
    prompt = f"""
    Estima el nivel de comprensión del estudiante en esta transcripción (Alto (80-100%), Medio (60-79%), Bajo (<60%)).
    Responde solo con el nivel.

    Transcripción: {transcripcion}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def generar_ticket_salida(transcripcion: str, tema: str) -> str:
    """Genera un ticket de salida usando Azure OpenAI, adaptado al tema."""
    prompt = f"""
    Genera 3 preguntas de ticket de salida relevantes para reforzar el aprendizaje en una sesión de mentoría sobre '{tema}'.
    Incluye una mezcla de preguntas de opción múltiple y abiertas.
    Formato: Pregunta 1\n[opciones si aplica]\n\nPregunta 2\n...\n\nPregunta 3\n...

    Basado en esta transcripción: {transcripcion[:1000]}  # Limitar longitud
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

def guardar_conversacion(transcripcion: str, tema: str, emocion: str, comprension: str, ticket: str) -> str:
    """Guarda en registro (simulado) y retorna log."""
    fecha = datetime.datetime.now().isoformat()
    registro = {
        "fecha": fecha,
        "tema": tema,
        "emocion": emocion,
        "comprension": comprension,
        "ticket": ticket,
        "transcripcion": transcripcion
    }
    global registros
    registros.append(registro)
    return f"[LOG] Conversación guardada (ID: {len(registros)}). Fecha: {fecha}"

# === Backend: Función Principal ===
def procesar_pregunta(pregunta: str):
    """Procesa la pregunta con RAG y luego clasifica la transcripción (pregunta + respuesta)."""
    if not pregunta.strip():
        return "Error: Ingresa una pregunta válida.", "", "", "", "", "[LOG] Error: Pregunta vacía."

    log = "[LOG] Iniciando procesamiento de pregunta...\n"

    # Paso 1: Búsqueda en Azure Search (Retrieval)
    contexto = search_documents(pregunta)
    log += f"[LOG] Paso 1: Búsqueda completada. Contexto recuperado: {len(contexto)} caracteres.\n"

    # Paso 2: Generar respuesta con RAG (Generation)
    respuesta = generate_answer(pregunta, contexto)
    log += "[LOG] Paso 2: Respuesta generada vía Azure OpenAI.\n"

    # Crear transcripción para clasificación: pregunta + respuesta
    transcripcion = f"Pregunta del usuario: {pregunta}\nRespuesta del mentor: {respuesta}"

    # Paso 3: Clasificar tema
    tema = clasificar_tema(transcripcion)
    log += f"[LOG] Paso 3: Tema clasificado como '{tema}' vía Azure OpenAI.\n"

    # Paso 4: Detectar emoción
    emocion = detectar_emocion(transcripcion)
    log += f"[LOG] Paso 4: Emoción detectada como '{emocion}' vía Azure OpenAI.\n"

    # Paso 5: Estimar comprensión
    comprension = estimar_comprension(transcripcion)
    log += f"[LOG] Paso 5: Nivel de comprensión estimado en '{comprension}' vía Azure OpenAI.\n"

    # Paso 6: Generar ticket de salida
    ticket = generar_ticket_salida(transcripcion, tema)
    log += f"[LOG] Paso 6: Ticket de salida generado para '{tema}'.\n"

    # Paso 7: Guardar conversación
    save_log = guardar_conversacion(transcripcion, tema, emocion, comprension, ticket)
    log += save_log + "\n"

    log += "[LOG] Procesamiento completado exitosamente con Azure."

    return respuesta, tema, emocion, comprension, ticket, log

# === Inicialización del Registro ===
registros = []

# === Frontend: Interfaz Gradio ===
iface = gr.Interface(
    fn=procesar_pregunta,
    inputs=gr.Textbox(lines=5, label="Ingresa tu pregunta sobre orientación vocacional", placeholder="Ejemplo: ¿Qué es Proyectate?"),
    outputs=[
        gr.Textbox(lines=10, label="Respuesta del RAG (Azure Search + OpenAI)"),
        gr.Textbox(label="Tema identificado (Clasificación)"),
        gr.Textbox(label="Emoción detectada (Clasificación)"),
        gr.Textbox(label="Nivel de comprensión (Clasificación)"),
        gr.Textbox(lines=10, label="Ticket de salida generado (Clasificación)"),
        gr.Textbox(lines=5, label="Logs del sistema (Backend con Azure)")
    ],
    title="Chat Vocacional con RAG y Clasificación (Azure)",
    description="Ingresa una pregunta para obtener una respuesta del sistema RAG (búsqueda en Azure Search + generación con Azure OpenAI). Luego, el sistema clasificará la conversación (pregunta + respuesta) en tema, emoción, comprensión y generará un ticket de salida. Los logs muestran el proceso backend paso a paso."
)

# Lanzar la app en Colab
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9e5a8cbd85138e9873.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4e700b124994b231d9.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://9e5a8cbd85138e9873.gradio.live


```markdown
# Código Markdown: Chat Vocacional con RAG Semántico y Clasificación (Azure)

## Instalación de Dependencias en Colab
```bash
!pip install gradio python-dotenv openai requests
```

## Importación de Módulos
```python
import gradio as gr
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests
import datetime
import json  # Para depuración de respuestas JSON
```

## Configuración de Azure (Search y OpenAI)
```python
# Carga variables de .env (sube tu archivo .env a Colab)
load_dotenv()

# Validar variables requeridas
required_vars = [
    "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
    "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
]
for var in required_vars:
    if not os.environ.get(var):
        raise ValueError(f"Falta variable de entorno: {var}")

# Configuración de Azure Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Inicializar cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2023-08-01-preview",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)
```

## Función de Búsqueda en Azure Search (RAG Retrieval) - Mejorada con Semántica y Depuración
```python
def search_documents(query: str, top_k: int = 5) -> str:
    """Realiza una búsqueda semántica en Azure Cognitive Search y retorna el contexto concatenado."""
    try:
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
        headers = {
            "Content-Type": "application/json",
            "api-key": AZURE_SEARCH_API_KEY
        }
        payload = {
            "search": query,
            "top": top_k,
            "queryType": "semantic",  # Habilitar búsqueda semántica para mejores resultados
            "semanticConfiguration": "default",  # Asumir config semántica 'default'; ajusta si es diferente
            "select": "content,source,@search.score"  # Seleccionar campos relevantes
        }
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        results = response.json()
        
        # Depuración: Imprimir resultados crudos para logs (opcional, remueve en prod)
        print(f"[DEBUG] Respuesta de Azure Search: {json.dumps(results, indent=2)[:500]}...")  # Primeros 500 chars
        
        documents = [doc.get("content", "") for doc in results.get("value", [])]
        contexto = "\n\n".join(documents)
        if not contexto.strip():
            return "No se encontraron documentos relevantes. Verifica el índice o reformula la consulta."
        return contexto
    except requests.exceptions.HTTPError as e:
        error_msg = f"Error HTTP en búsqueda: {e.response.status_code} - {e.response.text[:200]}"
        print(f"[ERROR] {error_msg}")
        return f"Error en búsqueda: {error_msg}"
    except Exception as e:
        error_msg = f"Error inesperado en búsqueda: {str(e)}"
        print(f"[ERROR] {error_msg}")
        return f"Error en búsqueda: {error_msg}"
```

## Función de Generación de Respuesta (RAG Generation)
```python
def generate_answer(question: str, context: str) -> str:
    """Genera una respuesta usando Azure OpenAI basada en pregunta y contexto de búsqueda."""
    if not context or "No se encontraron" in context:
        return "No hay contexto suficiente para responder con precisión. Intenta reformular la pregunta o verifica que el índice de Azure Search tenga datos relevantes sobre orientación vocacional."
    system_prompt = """
    Eres un mentor pedagógico especializado en orientación vocacional y educación en Chile.
    Responde de forma clara, breve, útil y empática, basándote únicamente en el contexto proporcionado.
    Si el contexto es insuficiente o no contiene información relevante, indica que no puedes responder con precisión y sugiere al usuario proporcionar más detalles o contactar instituciones oficiales como el Mineduc.
    Estructura la respuesta con pasos accionables si aplica.
    """
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context[:2000]}"  # Limitar contexto para tokens
    try:
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error al generar respuesta: {str(e)}"
```

## Backend: Funciones de Clasificación con Azure OpenAI
### Clasificar Tema
```python
def clasificar_tema(transcripcion: str) -> str:
    """Clasifica el tema usando Azure OpenAI."""
    prompt = f"""
    Clasifica el tema principal de esta transcripción de una sesión de mentoría en una de estas categorías:
    Biología, Matemáticas, Lenguaje, Historia, Orientación Vocacional, Educación Superior, Otros.
    Responde solo con el nombre de la categoría.
   
    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()
```

### Detectar Emoción
```python
def detectar_emocion(transcripcion: str) -> str:
    """Detecta la emoción usando Azure OpenAI."""
    prompt = f"""
    Detecta la emoción predominante en esta transcripción de mentoría (Entusiasmo, Confusión, Curiosidad, Frustración, Motivación, Ansiedad).
    Responde solo con el nombre de la emoción.
   
    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()
```

### Estimar Comprensión
```python
def estimar_comprension(transcripcion: str) -> str:
    """Estima el nivel de comprensión usando Azure OpenAI."""
    prompt = f"""
    Estima el nivel de comprensión del estudiante en esta transcripción (Alto (80-100%), Medio (60-79%), Bajo (<60%)).
    Responde solo con el nivel.
   
    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()
```

### Generar Ticket de Salida
```python
def generar_ticket_salida(transcripcion: str, tema: str) -> str:
    """Genera un ticket de salida usando Azure OpenAI, adaptado al tema."""
    prompt = f"""
    Genera 3 preguntas de ticket de salida relevantes para reforzar el aprendizaje en una sesión de mentoría sobre '{tema}' en contexto chileno (e.g., Mineduc, crédito universitario).
    Incluye una mezcla de preguntas de opción múltiple y abiertas.
    Formato: Pregunta 1\n[opciones si aplica]\n\nPregunta 2\n...\n\nPregunta 3\n...
   
    Basado en esta transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()
```

### Guardar Conversación
```python
def guardar_conversacion(transcripcion: str, tema: str, emocion: str, comprension: str, ticket: str) -> str:
    """Guarda en registro (simulado) y retorna log."""
    fecha = datetime.datetime.now().isoformat()
    registro = {
        "fecha": fecha,
        "tema": tema,
        "emocion": emocion,
        "comprension": comprension,
        "ticket": ticket,
        "transcripcion": transcripcion
    }
    global registros
    registros.append(registro)
    return f"[LOG] Conversación guardada (ID: {len(registros)}). Fecha: {fecha}"
```

## Backend: Función Principal
```python
def procesar_pregunta(pregunta: str):
    """Procesa la pregunta con RAG (semántico) y luego clasifica la transcripción (pregunta + respuesta)."""
    if not pregunta.strip():
        return "Error: Ingresa una pregunta válida.", "", "", "", "", "[LOG] Error: Pregunta vacía."
    log = "[LOG] Iniciando procesamiento de pregunta...\n"
   
    # Paso 1: Búsqueda semántica en Azure Search (Retrieval)
    contexto = search_documents(pregunta)
    log += f"[LOG] Paso 1: Búsqueda semántica completada. Contexto recuperado: {len(contexto)} caracteres.\n"
    if "No se encontraron" in contexto:
        log += "[LOG] ADVERTENCIA: Búsqueda sin resultados. Verifica el índice en Azure Portal (poblado con chunks de documentos vocacionales).\n"
   
    # Paso 2: Generar respuesta con RAG (Generation)
    respuesta = generate_answer(pregunta, contexto)
    log += "[LOG] Paso 2: Respuesta generada vía Azure OpenAI.\n"
   
    # Crear transcripción para clasificación: pregunta + respuesta
    transcripcion = f"Pregunta del usuario: {pregunta}\nRespuesta del mentor: {respuesta}"
   
    # Paso 3: Clasificar tema
    tema = clasificar_tema(transcripcion)
    log += f"[LOG] Paso 3: Tema clasificado como '{tema}' vía Azure OpenAI.\n"
   
    # Paso 4: Detectar emoción
    emocion = detectar_emocion(transcripcion)
    log += f"[LOG] Paso 4: Emoción detectada como '{emocion}' vía Azure OpenAI.\n"
   
    # Paso 5: Estimar comprensión
    comprension = estimar_comprension(transcripcion)
    log += f"[LOG] Paso 5: Nivel de comprensión estimado en '{comprension}' vía Azure OpenAI.\n"
   
    # Paso 6: Generar ticket de salida
    ticket = generar_ticket_salida(transcripcion, tema)
    log += f"[LOG] Paso 6: Ticket de salida generado para '{tema}'.\n"
   
    # Paso 7: Guardar conversación
    save_log = guardar_conversacion(transcripcion, tema, emocion, comprension, ticket)
    log += save_log + "\n"
   
    log += "[LOG] Procesamiento completado exitosamente con Azure (búsqueda semántica activada)."
   
    return respuesta, tema, emocion, comprension, ticket, log
```

## Inicialización del Registro
```python
registros = []
```

## Frontend: Interfaz Gradio
```python
iface = gr.Interface(
    fn=procesar_pregunta,
    inputs=gr.Textbox(lines=5, label="Ingresa tu pregunta sobre orientación vocacional", placeholder="Ejemplo: ¿Qué debo hacer si perdí mis beneficios del crédito universitario?"),
    outputs=[
        gr.Textbox(lines=10, label="Respuesta del RAG (Búsqueda Semántica + OpenAI)"),
        gr.Textbox(label="Tema identificado (Clasificación)"),
        gr.Textbox(label="Emoción detectada (Clasificación)"),
        gr.Textbox(label="Nivel de comprensión (Clasificación)"),
        gr.Textbox(lines=10, label="Ticket de salida generado (Clasificación)"),
        gr.Textbox(lines=7, label="Logs del sistema (Backend con Azure - Incluye Debug)")
    ],
    title="Chat Vocacional con RAG Semántico y Clasificación (Azure)",
    description="Ingresa una pregunta para obtener una respuesta mejorada del sistema RAG (búsqueda semántica en Azure Search + generación con Azure OpenAI). Luego, clasifica la conversación. Logs incluyen debug para depurar búsquedas vacías."
)
```

## Lanzar la App en Colab
```python
iface.launch(share=True, debug=True)
```
```

In [6]:
# Instalación de dependencias en Colab
#!pip install gradio python-dotenv openai requests

import gradio as gr
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests
import datetime
import json  # Para depuración de respuestas JSON

# === Configuración de Azure (Search y OpenAI) ===
load_dotenv()  # Carga variables de .env (sube tu archivo .env a Colab)

# Validar variables requeridas
required_vars = [
    "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
    "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
]
for var in required_vars:
    if not os.environ.get(var):
        raise ValueError(f"Falta variable de entorno: {var}")

# Configuración de Azure Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Inicializar cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2023-08-01-preview",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

# === Función de Búsqueda en Azure Search (RAG Retrieval) - Mejorada con Semántica y Depuración ===
def search_documents(query: str, top_k: int = 5) -> str:
    """Realiza una búsqueda semántica en Azure Cognitive Search y retorna el contexto concatenado."""
    try:
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
        headers = {
            "Content-Type": "application/json",
            "api-key": AZURE_SEARCH_API_KEY
        }
        payload = {
            "search": query,
            "top": top_k,
            "queryType": "semantic",  # Habilitar búsqueda semántica para mejores resultados
            "semanticConfiguration": "default",  # Asumir config semántica 'default'; ajusta si es diferente
            "select": "content,source,@search.score"  # Seleccionar campos relevantes
        }
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        results = response.json()

        # Depuración: Imprimir resultados crudos para logs (opcional, remueve en prod)
        print(f"[DEBUG] Respuesta de Azure Search: {json.dumps(results, indent=2)[:500]}...")  # Primeros 500 chars

        documents = [doc.get("content", "") for doc in results.get("value", [])]
        contexto = "\n\n".join(documents)
        if not contexto.strip():
            return "No se encontraron documentos relevantes. Verifica el índice o reformula la consulta."
        return contexto
    except requests.exceptions.HTTPError as e:
        error_msg = f"Error HTTP en búsqueda: {e.response.status_code} - {e.response.text[:200]}"
        print(f"[ERROR] {error_msg}")
        return f"Error en búsqueda: {error_msg}"
    except Exception as e:
        error_msg = f"Error inesperado en búsqueda: {str(e)}"
        print(f"[ERROR] {error_msg}")
        return f"Error en búsqueda: {error_msg}"

# === Función de Generación de Respuesta (RAG Generation) ===
def generate_answer(question: str, context: str) -> str:
    """Genera una respuesta usando Azure OpenAI basada en pregunta y contexto de búsqueda."""
    if not context or "No se encontraron" in context:
        return "No hay contexto suficiente para responder con precisión. Intenta reformular la pregunta o verifica que el índice de Azure Search tenga datos relevantes sobre orientación vocacional."

    system_prompt = """
    Eres un mentor pedagógico especializado en orientación vocacional y educación en Chile.
    Responde de forma clara, breve, útil y empática, basándote únicamente en el contexto proporcionado.
    Si el contexto es insuficiente o no contiene información relevante, indica que no puedes responder con precisión y sugiere al usuario proporcionar más detalles o contactar instituciones oficiales como el Mineduc.
    Estructura la respuesta con pasos accionables si aplica.
    """
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context[:2000]}"  # Limitar contexto para tokens
    try:
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error al generar respuesta: {str(e)}"

# === Backend: Funciones de Clasificación con Azure OpenAI ===
def clasificar_tema(transcripcion: str) -> str:
    """Clasifica el tema usando Azure OpenAI."""
    prompt = f"""
    Clasifica el tema principal de esta transcripción de una sesión de mentoría en una de estas categorías:
    Biología, Matemáticas, Lenguaje, Historia, Orientación Vocacional, Educación Superior, Otros.
    Responde solo con el nombre de la categoría.

    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def detectar_emocion(transcripcion: str) -> str:
    """Detecta la emoción usando Azure OpenAI."""
    prompt = f"""
    Detecta la emoción predominante en esta transcripción de mentoría (Entusiasmo, Confusión, Curiosidad, Frustración, Motivación, Ansiedad).
    Responde solo con el nombre de la emoción.

    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def estimar_comprension(transcripcion: str) -> str:
    """Estima el nivel de comprensión usando Azure OpenAI."""
    prompt = f"""
    Estima el nivel de comprensión del estudiante en esta transcripción (Alto (80-100%), Medio (60-79%), Bajo (<60%)).
    Responde solo con el nivel.

    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def generar_ticket_salida(transcripcion: str, tema: str) -> str:
    """Genera un ticket de salida usando Azure OpenAI, adaptado al tema."""
    prompt = f"""
    Genera 3 preguntas de ticket de salida relevantes para reforzar el aprendizaje en una sesión de mentoría sobre '{tema}' en contexto chileno (e.g., Mineduc, crédito universitario).
    Incluye una mezcla de preguntas de opción múltiple y abiertas.
    Formato: Pregunta 1\n[opciones si aplica]\n\nPregunta 2\n...\n\nPregunta 3\n...

    Basado en esta transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

def guardar_conversacion(transcripcion: str, tema: str, emocion: str, comprension: str, ticket: str) -> str:
    """Guarda en registro (simulado) y retorna log."""
    fecha = datetime.datetime.now().isoformat()
    registro = {
        "fecha": fecha,
        "tema": tema,
        "emocion": emocion,
        "comprension": comprension,
        "ticket": ticket,
        "transcripcion": transcripcion
    }
    global registros
    registros.append(registro)
    return f"[LOG] Conversación guardada (ID: {len(registros)}). Fecha: {fecha}"

# === Backend: Función Principal ===
def procesar_pregunta(pregunta: str):
    """Procesa la pregunta con RAG (semántico) y luego clasifica la transcripción (pregunta + respuesta)."""
    if not pregunta.strip():
        return "Error: Ingresa una pregunta válida.", "", "", "", "", "[LOG] Error: Pregunta vacía."

    log = "[LOG] Iniciando procesamiento de pregunta...\n"

    # Paso 1: Búsqueda semántica en Azure Search (Retrieval)
    contexto = search_documents(pregunta)
    log += f"[LOG] Paso 1: Búsqueda semántica completada. Contexto recuperado: {len(contexto)} caracteres.\n"
    if "No se encontraron" in contexto:
        log += "[LOG] ADVERTENCIA: Búsqueda sin resultados. Verifica el índice en Azure Portal (poblado con chunks de documentos vocacionales).\n"

    # Paso 2: Generar respuesta con RAG (Generation)
    respuesta = generate_answer(pregunta, contexto)
    log += "[LOG] Paso 2: Respuesta generada vía Azure OpenAI.\n"

    # Crear transcripción para clasificación: pregunta + respuesta
    transcripcion = f"Pregunta del usuario: {pregunta}\nRespuesta del mentor: {respuesta}"

    # Paso 3: Clasificar tema
    tema = clasificar_tema(transcripcion)
    log += f"[LOG] Paso 3: Tema clasificado como '{tema}' vía Azure OpenAI.\n"

    # Paso 4: Detectar emoción
    emocion = detectar_emocion(transcripcion)
    log += f"[LOG] Paso 4: Emoción detectada como '{emocion}' vía Azure OpenAI.\n"

    # Paso 5: Estimar comprensión
    comprension = estimar_comprension(transcripcion)
    log += f"[LOG] Paso 5: Nivel de comprensión estimado en '{comprension}' vía Azure OpenAI.\n"

    # Paso 6: Generar ticket de salida
    ticket = generar_ticket_salida(transcripcion, tema)
    log += f"[LOG] Paso 6: Ticket de salida generado para '{tema}'.\n"

    # Paso 7: Guardar conversación
    save_log = guardar_conversacion(transcripcion, tema, emocion, comprension, ticket)
    log += save_log + "\n"

    log += "[LOG] Procesamiento completado exitosamente con Azure (búsqueda semántica activada)."

    return respuesta, tema, emocion, comprension, ticket, log

# === Inicialización del Registro ===
registros = []

# === Frontend: Interfaz Gradio ===
iface = gr.Interface(
    fn=procesar_pregunta,
    inputs=gr.Textbox(lines=5, label="Ingresa tu pregunta sobre orientación vocacional", placeholder="Ejemplo: ¿Qué debo hacer si perdí mis beneficios del crédito universitario?"),
    outputs=[
        gr.Textbox(lines=10, label="Respuesta del RAG (Búsqueda Semántica + OpenAI)"),
        gr.Textbox(label="Tema identificado (Clasificación)"),
        gr.Textbox(label="Emoción detectada (Clasificación)"),
        gr.Textbox(label="Nivel de comprensión (Clasificación)"),
        gr.Textbox(lines=10, label="Ticket de salida generado (Clasificación)"),
        gr.Textbox(lines=7, label="Logs del sistema (Backend con Azure - Incluye Debug)")
    ],
    title="Chat Vocacional con RAG Semántico y Clasificación (Azure)",
    description="Ingresa una pregunta para obtener una respuesta mejorada del sistema RAG (búsqueda semántica en Azure Search + generación con Azure OpenAI). Luego, clasifica la conversación. Logs incluyen debug para depurar búsquedas vacías."
)

# Lanzar la app en Colab
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4fb34f0c7bcaebeba3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[ERROR] Error HTTP en búsqueda: 400 - {"error":{"code":"","message":"Invalid expression: Could not find a property named 'content' on type 'search.document'.\r\nParameter name: $select"}}
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://4fb34f0c7bcaebeba3.gradio.live


In [7]:
import gradio as gr
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests
import datetime
import json
from functools import lru_cache  # Para caché simple

# === Configuración de Azure (Search y OpenAI) ===
load_dotenv()  # Carga variables de .env

# Validar variables requeridas
required_vars = [
    "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
    "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
]
for var in required_vars:
    if not os.environ.get(var):
        raise ValueError(f"Falta variable de entorno: {var}")

# Configuración de Azure Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Inicializar cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2023-08-01-preview",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

# === Función para Obtener Esquema del Índice (Nueva para Depuración) ===
def get_index_schema():
    """Obtiene los campos disponibles en el índice de Azure Search."""
    try:
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}?api-version=2023-07-01-Preview"
        headers = {"api-key": AZURE_SEARCH_API_KEY}
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        schema = response.json()
        fields = schema.get("fields", [])
        searchable_fields = [field["name"] for field in fields if field.get("searchable", False)]
        print(f"[DEBUG] Campos disponibles en el índice (buscables): {searchable_fields}")
        return searchable_fields
    except Exception as e:
        print(f"[ERROR] No se pudo obtener el esquema: {str(e)}")
        return []

# Obtener esquema al inicio para depuración
available_fields = get_index_schema()

# Campo por defecto para contenido; ajusta si tu índice usa otro (ej. 'text' o 'description')
DEFAULT_CONTENT_FIELD = "content"  # Cambia esto si es necesario, basado en available_fields

# === Función de Búsqueda en Azure Search (Mejorada con Semántica y Manejo de Errores) ===
@lru_cache(maxsize=100)  # Caché simple para evitar búsquedas repetidas
def search_documents(query: str, top_k: int = 5) -> str:
    """Realiza una búsqueda semántica en Azure Cognitive Search y retorna el contexto concatenado."""
    try:
        # Usar campos válidos dinámicamente
        content_field = next((f for f in [DEFAULT_CONTENT_FIELD, "text", "description"] if f in available_fields), None)
        if not content_field:
            return "Error: No se encontró un campo de contenido válido en el índice. Revisa available_fields en logs y ajusta DEFAULT_CONTENT_FIELD."

        select_fields = [content_field, "source", "@search.score"]
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
        headers = {
            "Content-Type": "application/json",
            "api-key": AZURE_SEARCH_API_KEY
        }
        payload = {
            "search": query,
            "top": top_k,
            "queryType": "semantic",
            "semanticConfiguration": "default",  # Ajusta si usas una config personalizada
            "select": ",".join(select_fields),
            "filter": "category eq 'beneficios_estudiantiles'"  # Filtro opcional; ajusta o elimina si no aplica
        }
        print(f"[DEBUG] Payload enviado a Azure Search: {json.dumps(payload, indent=2)[:300]}...")  # Log parcial para evitar spam

        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        results = response.json()

        print(f"[DEBUG] Respuesta de Azure Search (resumen): {len(results.get('value', []))} documentos encontrados.")

        documents = [doc.get(content_field, "") for doc in results.get("value", [])]
        contexto = "\n\n".join(documents)
        if not contexto.strip():
            return "No se encontraron documentos relevantes para esta consulta. Verifica el índice o reformula la pregunta."
        return contexto
    except requests.exceptions.HTTPError as e:
        error_msg = f"Error HTTP en búsqueda: {e.response.status_code} - {e.response.text[:200]}"
        print(f"[ERROR] {error_msg}")
        return f"Error en búsqueda: {error_msg}"
    except Exception as e:
        error_msg = f"Error inesperado en búsqueda: {str(e)}"
        print(f"[ERROR] {error_msg}")
        return f"Error en búsqueda: {error_msg}"

# === Función de Generación de Respuesta (Actualizada con Mensaje de Fallback) ===
def generate_answer(question: str, context: str) -> str:
    """Genera una respuesta usando Azure OpenAI basada en pregunta y contexto de búsqueda."""
    if not context or "No se encontraron" in context or len(context) < 100:  # Umbral para contexto insuficiente
        return "No pudimos recuperar esa respuesta. Escribe a contacto@innovacien.org si sigues teniendo dudas."

    system_prompt = """
    Eres un mentor pedagógico especializado en orientación vocacional y beneficios estudiantiles en Chile (becas, créditos, subvenciones del Mineduc, etc.).
    Responde de forma clara, breve, útil y empática, basándote únicamente en el contexto proporcionado.
    Estructura la respuesta con pasos accionables si aplica (ej. contactar Mineduc, revisar DEMRE).
    Si el contexto es insuficiente o no contiene información relevante, responde: "No pudimos recuperar esa respuesta. Escribe a contacto@innovacien.org si sigues teniendo dudas."
    """
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context[:1500]}"  # Limitar para tokens
    try:
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error al generar respuesta: {str(e)}"

# === Funciones de Clasificación (Sin Cambios, pero con límite de tokens) ===
def clasificar_tema(transcripcion: str) -> str:
    """Clasifica el tema usando Azure OpenAI."""
    prompt = f"""
    Clasifica el tema principal de esta transcripción de una sesión de mentoría en una de estas categorías:
    Biología, Matemáticas, Lenguaje, Historia, Orientación Vocacional, Educación Superior, Beneficios Estudiantiles, Otros.
    Responde solo con el nombre de la categoría.

    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def detectar_emocion(transcripcion: str) -> str:
    """Detecta la emoción usando Azure OpenAI."""
    prompt = f"""
    Detecta la emoción predominante en esta transcripción de mentoría (Entusiasmo, Confusión, Curiosidad, Frustración, Motivación, Ansiedad).
    Responde solo con el nombre de la emoción.

    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def estimar_comprension(transcripcion: str) -> str:
    """Estima el nivel de comprensión usando Azure OpenAI."""
    prompt = f"""
    Estima el nivel de comprensión del estudiante en esta transcripción (Alto (80-100%), Medio (60-79%), Bajo (<60%)).
    Responde solo con el nivel.

    Transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    return response.choices[0].message.content.strip()

def generar_ticket_salida(transcripcion: str, tema: str) -> str:
    """Genera un ticket de salida usando Azure OpenAI, adaptado al tema."""
    prompt = f"""
    Genera 3 preguntas de ticket de salida relevantes para reforzar el aprendizaje en una sesión de mentoría sobre '{tema}' en contexto chileno (e.g., Mineduc, crédito universitario).
    Incluye una mezcla de preguntas de opción múltiple y abiertas.
    Formato: Pregunta 1\n[opciones si aplica]\n\nPregunta 2\n...\n\nPregunta 3\n...

    Basado en esta transcripción: {transcripcion[:1000]}
    """
    response = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

def guardar_conversacion(transcripcion: str, tema: str, emocion: str, comprension: str, ticket: str) -> str:
    """Guarda en registro (simulado) y retorna log."""
    fecha = datetime.datetime.now().isoformat()
    registro = {
        "fecha": fecha,
        "tema": tema,
        "emocion": emocion,
        "comprension": comprension,
        "ticket": ticket,
        "transcripcion": transcripcion
    }
    global registros
    registros.append(registro)
    return f"[LOG] Conversación guardada (ID: {len(registros)}). Fecha: {fecha}"

# === Backend: Función Principal (Mejorada con Validación) ===
def procesar_pregunta(pregunta: str):
    """Procesa la pregunta con RAG (semántico) y luego clasifica la transcripción (pregunta + respuesta)."""
    if not pregunta.strip():
        return "Error: Ingresa una pregunta válida.", "", "", "", "", "[LOG] Error: Pregunta vacía."

    log = "[LOG] Iniciando procesamiento de pregunta...\n"

    # Validación simple para relevancia
    if any(palabra in pregunta.lower() for palabra in ["beneficios", "estudiantiles", "beca", "crédito"]):
        log += "[LOG] Pregunta relacionada con beneficios estudiantiles detectada.\n"
    else:
        log += "[ADVERTENCIA] La pregunta podría no estar enfocada en beneficios estudiantiles.\n"

    # Paso 1: Búsqueda semántica en Azure Search (Retrieval)
    contexto = search_documents(pregunta)
    log += f"[LOG] Paso 1: Búsqueda semántica completada. Contexto recuperado: {len(contexto)} caracteres.\n"
    if "No se encontraron" in contexto or "Error" in contexto:
        log += "[LOG] ADVERTENCIA: Búsqueda sin resultados o con error. Usando fallback en generación.\n"

    # Paso 2: Generar respuesta con RAG (Generation)
    respuesta = generate_answer(pregunta, contexto)
    log += "[LOG] Paso 2: Respuesta generada vía Azure OpenAI.\n"

    # Crear transcripción para clasificación: pregunta + respuesta
    transcripcion = f"Pregunta del usuario: {pregunta}\nRespuesta del mentor: {respuesta}"

    # Paso 3: Clasificar tema
    tema = clasificar_tema(transcripcion)
    log += f"[LOG] Paso 3: Tema clasificado como '{tema}' vía Azure OpenAI.\n"

    # Paso 4: Detectar emoción
    emocion = detectar_emocion(transcripcion)
    log += f"[LOG] Paso 4: Emoción detectada como '{emocion}' vía Azure OpenAI.\n"

    # Paso 5: Estimar comprensión
    comprension = estimar_comprension(transcripcion)
    log += f"[LOG] Paso 5: Nivel de comprensión estimado en '{comprension}' vía Azure OpenAI.\n"

    # Paso 6: Generar ticket de salida
    ticket = generar_ticket_salida(transcripcion, tema)
    log += f"[LOG] Paso 6: Ticket de salida generado para '{tema}'.\n"

    # Paso 7: Guardar conversación
    save_log = guardar_conversacion(transcripcion, tema, emocion, comprension, ticket)
    log += save_log + "\n"

    log += "[LOG] Procesamiento completado exitosamente con Azure (búsqueda semántica activada)."

    return respuesta, tema, emocion, comprension, ticket, log

# === Inicialización del Registro ===
registros = []

# === Frontend: Interfaz Gradio ===
iface = gr.Interface(
    fn=procesar_pregunta,
    inputs=gr.Textbox(lines=5, label="Ingresa tu pregunta sobre orientación vocacional o beneficios estudiantiles", placeholder="Ejemplo: ¿Qué debo hacer si perdí mis beneficios del crédito universitario?"),
    outputs=[
        gr.Textbox(lines=10, label="Respuesta del RAG (Búsqueda Semántica + OpenAI)"),
        gr.Textbox(label="Tema identificado (Clasificación)"),
        gr.Textbox(label="Emoción detectada (Clasificación)"),
        gr.Textbox(label="Nivel de comprensión (Clasificación)"),
        gr.Textbox(lines=10, label="Ticket de salida generado (Clasificación)"),
        gr.Textbox(lines=7, label="Logs del sistema (Backend con Azure - Incluye Debug)")
    ],
    title="Chat Vocacional con RAG Semántico y Clasificación (Azure) - Versión Mejorada",
    description="Ingresa una pregunta para obtener una respuesta mejorada. Si no hay datos suficientes, se sugiere contactar a contacto@innovacien.org. Logs incluyen esquema del índice para depuración."
)

# Lanzar la app
iface.launch(share=True, debug=True)

[ERROR] No se pudo obtener el esquema: 403 Client Error: Forbidden for url: https://search-vocatest.search.windows.net/indexes/rag-vocacional?api-version=2023-07-01-Preview
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://80bb2f24f801537d72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://80bb2f24f801537d72.gradio.live
